In [1]:
from pyspark.sql import SparkSession
import findspark

findspark.init()

In [5]:
spark = SparkSession\
    .builder\
    .appName('gpsTrackingSystem')\
    .enableHiveSupport()\
    .master('local')\
    .getOrCreate()

sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local appName=gpsTrackingSystem>

Bad pipe message: %s [b'" Not A;Brand";v="99", "Chromium";v="100", "Opera GX";v="']
Bad pipe message: %s [b'"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "macOS"\r\nU']
Bad pipe message: %s [b'rade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTM', b' like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.64 (Edition ']
Bad pipe message: %s [b'd-1)\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,app']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: " Not A;Brand";v="99", "Chromium";v="100", "Opera GX";v="86"\r\nsec-ch-ua-mobile: ?0\r\nsec-c']
Bad pipe message: %s [b'ua-platform: "macOS"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X ', b'_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Ch']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: " Not A;Brand";v="99", "Chromium";v="100", "Opera GX";v="86"\r\nsec-ch-ua-mobi